<a href="https://colab.research.google.com/github/gdapriana/machine-learning/blob/main/capstone/knn_nearest_destinations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data:

- Tourism destinations: List of tourist attractions with name, description, category, website, photos, and coordinates (latitude, longitude).
- User location: User's current location coordinates (latitude, longitude).

In [1]:
from urllib import request
import tensorflow as tf
import pandas as pd


In [2]:
data_url = "https://raw.githubusercontent.com/Touventure/models/main/sample-data/places-of-tourist-destinations-in-bali.csv"
request.urlretrieve(data_url, "dataset.csv")
df = pd.read_csv('dataset.csv')
df.head(3)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 57 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   place_id                  45 non-null     object 
 1   name                      45 non-null     object 
 2   description               27 non-null     object 
 3   is_spending_on_ads        45 non-null     bool   
 4   competitors               44 non-null     object 
 5   reviews                   45 non-null     int64  
 6   website                   26 non-null     object 
 7   owner_name                45 non-null     object 
 8   owner_profile_link        35 non-null     object 
 9   featured_image            45 non-null     object 
 10  main_category             45 non-null     object 
 11  categories                45 non-null     object 
 12  rating                    45 non-null     float64
 13  workday_timing            36 non-null     object 
 14  closed_on   

In [3]:
df = df[['place_id', 'name', 'coordinates']]
df[["latitude", "longitude"]] = df["coordinates"].str.split(",", expand=True)
df = df.drop_duplicates()
df = df.dropna()
df.head()

<ipython-input-3-bcacc1568589>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["latitude", "longitude"]] = df["coordinates"].str.split(",", expand=True)
<ipython-input-3-bcacc1568589>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["latitude", "longitude"]] = df["coordinates"].str.split(",", expand=True)


,place_id,name,coordinates,latitude,longitude
0,ChIJBzFesyFb0i0RhIzFAVTEb3c,BALI HELITOUR,"-8.808322,115.1625794",-8.808322,115.1625794
1,ChIJ14CrqGJD0i0RiLYCqHQJ0XA,Bali hire driver,"-8.7841086,115.1890555",-8.7841086,115.1890555
2,ChIJq95xT4I30i0RaU3j93Diq8o,Tanah Lot,"-8.621213,115.086807",-8.621213,115.086807
3,ChIJx97hVM9E0i0RUVpPBj5miBk,Garuda Wisnu Kencana Cultural Park,"-8.8104228,115.16759859999999",-8.8104228,115.16759859999999
4,ChIJZ5sY9kM90i0RVsPvSxZD7LY,Sacred Monkey Forest Sanctuary,"-8.5193727,115.2606299",-8.5193727,115.2606299


In [4]:
tensor = tf.convert_to_tensor(df[['latitude', 'longitude']].values, dtype=tf.float32)

In [5]:
class KNN:
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def predict(self, X, K=10):
        r = tf.reduce_sum(X**2, axis=1) - 2 * tf.matmul(X, tf.transpose(self.X)) + tf.reduce_sum(self.X**2, axis=1)
        distances = tf.sqrt(r)
        _, indices = tf.nn.top_k(tf.negative(distances), k=K)
        return tf.gather(self.y, indices)

In [28]:
model = KNN(tensor, df[['place_id', 'name']])

In [34]:
new_point = tf.constant([[-8.574264790858729, 115.34931244326434]], dtype=tf.float32)
predictions = model.predict(new_point)
predictions = predictions.numpy().tolist()[0]
data = [[place_id.decode(), name.decode()] for place_id, name in predictions]
predictions_df = pd.DataFrame(data, columns=['place_id', 'name'])

predictions_df.head(10)

,place_id,name
0,ChIJoQHcmv8X0i0RI6vXSuiyCOo,Kanto Lampo Waterfall
1,ChIJ723cqmwR0i0RlxUQK--WAks,Klungkung Royal Palace
2,ChIJYQy97hsW0i0RXgxBuN15rsE,Tegenungan Waterfall
3,ChIJ93viDd0X0i0RokCC4Z_TgDo,Sumampan Waterfall
4,ChIJv_tYCBIX0i0Rn_MGS2i_Kpc,Goa Rang Reng Waterfall
5,ChIJLRhilH4V0i0Rez5UbFcCNm4,Hidden Canyon Beji Guwang
6,ChIJQQrhbjUX0i0ReTctOJsyMLo,Taman Sari Waterfall & Natural pool
7,ChIJRSElPXYR0i0R-l62epaUmSw,Bukit Belong
8,ChIJa3Khjgw90i0Rlz1pgcp3WeY,Adi Bali Tour Official
9,ChIJgxNvhqEX0i0R-6U9ogQJ_-k,Suwat Waterfall
